<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT19/blob/bozzato/Lab-2/bozzato_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Direct methods**
**Bozzato Federico**

# **Abstract**

In this lab we will see how these general methods can be used to solve algebraic problems, such as to find the inverse of a matrix or to compute the eigenvalues of a certain matrix, which are theoretically simple to solve but very hard to compute.

#**About the code**

**Author:** Federico Bozzato 

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import sys
import math
import numpy as np
from numpy import linalg

# **Introduction**

Direct methods are general methods for constructing a proof of the existence of a minimizer for a given functional [1]. This kind of methods are widely used to solve several algebraic problems, which may be computationally difficult to implement: for example to find eigenvalues and eigenvectors of an $n\times n$ matrix with $n \ge 4$ is a non trivial problem to solve.

Even though some of these methods can give us approximate results, for particular types of inputs these methods reach high performances. In fact, in addition to being used to prove the existence of a solution,  direct methods may be used to compute the solution to derised accurancy [1].


This report is divided into three parts:

1. Methods: in this section, each algorithm is presented and explained, giving also the mathematic definition of the operation the algorithm implements

2. Results: in this section, the results of each algorithm are presented.

3. Discussion: in this last section, results are discussed.

# **Methods**

This section is divided into two subsection: the mandatory part and the extra assignment.

## Mandatory assignment

###1. QR factorization
####Definition
QR factorization is a method used to solve a generic equation such as $Ax=b$ where $A$ is an $n\times n$ matrix, $b$ is an $n\times 1$ vector and $x$ is the unknown vector. 

The key idea of QR factorization consists of decomposing an $n\times n$ matrix $A$ into the product $A=QR$, where $Q$ is an orthogonal matrix (which means $Q^{-1}=Q^{T}$) and $R$ is an upper triangular matrix [2].

####Implementation
One way to implement QR factorization is to start from Gram-Schmidt algorithm, that orthogonalizes vectors belonging to the same vector space $V$. 

Given a square matrix $A\in\mathbb{R}^{n\times n}$, with its columns defining the vectors $\{a_{i}\}_{i=1}^n$, the new orthogonal vectors are given by [3]

>$
v_j = a_j - \displaystyle \sum_{i=1}^{j-1} \left(a_j,q_i\right)q_i \qquad \forall j=1,\dots,n
$

where

* $a_j$ is the $j-$th column of matrix $A$
* $q_i$ is the normalized vector $q_j= \frac{v_i}{||v_i||}$

These equations can be rewritten in the following way

>$
  \begin{align*}
    a_1 &= r_{11}q_1 \\
    a_2 &= r_{12}q_1 + r_{22}q_2 \\
           &\vdots\\
    a_n &= r_{1n}q_1 + \dots + r_{nn}q_n
  \end{align*}
$

where

* $r_{ij}=(a_j,q_i)$
* $r_{jj}=||a_j-\sum_{i=1}^{j-1}(a_j,q_i)q_i||$

As it is possible to observe, the rewritten equations corresponds to product $A=QR$, where the columns of $Q$ are given by ${q_i}_{i=1}^n$ and $R$ is an uppen triangular matrix.

The algorithm implementing the QR factorization is the following one [4]:


```
for i = 1 to n do
  v_i = a_i
  r_ii = abs(v_i)
  q_i= v_i / r_ii
  for j = i+1 to n do
    r_ij = transpose(q_i)v_j
    v_j = v_i - r_ij*q_i
  end
end
```



In [0]:
def QRfactorization(matrixA):
  '''
  Calculates the QR factorization of matrixA which must be invertible and 
  returns the orthogonal matrix Q and the upper triangular matrix R such that
  A = QR
  Parameters:
  - matrixA: square matrix to be factorized
  Output:
  - Q: orthogonal matrix
  - R: upper triangular matrix
  '''
  
  if not isinstance(matrixA,np.ndarray):
    matrixA= np.array(matrixA)
    
  if matrixA.shape[0] != matrixA.shape[1]:
    print('Error: only square matrices are allowed!')
    sys.exit(1)
    
  n= matrixA.shape[0]
  
  v= np.zeros((n,n))
  for i in range(0,n):
    v[:,i]= matrixA[:,i]
    
  Q= np.zeros((n,n))
  R= np.zeros((n,n))
  for i in range(0,n):
    r_ii = np.dot(v[:,i],v[:,i])**(0.5)
    
    R[i,i]= r_ii
    q_i= v[:,i]/r_ii
    Q[:,i]= q_i
    
    for j in range(i+1,n):
      r_ij= np.dot(q_i,v[:,j])
      R[i,j]= r_ij
      v[:,j]= v[:,j] - r_ij*q_i
  
  return Q, R

###2. Direct solver $Ax=b$
####Definition
Let $A$ and $b$ be an $n\times n$ matrix and $n\times 1$ vector, respectively. It is possible to solve the system of equations $Ax=b$ by using the QR factorization. In fact, $A$ can be decomposed into the product $A=QR$, where $Q$ is a orthogonal matrix and $R$ is an upper triangular matrix. Thus,

>$
  Ax= B \Longrightarrow QRx=b
$

Since $Q^{-1}= Q^{T}$ because $Q$ is orthogonal, we obtain

>$
Rx= Q^{T}b
$

Now, in order to get $x$, we can use the *backward substitution* explained in [5].


####Implementation
One of the possible implementations is the following one:

In [0]:
def linearSystemSolver(matrixA,b):
  '''
  Calculates the unknown vector x of the equation Ax=b.
  Parameters:
  - matrixA: n x n square matrix 
  - b: n x 1 vector
  Output:
  - x: n x 1 unknowm vector
  '''
  
  if not isinstance(matrixA,np.ndarray):
    matrixA= np.array(matrixA)
    
  if not isinstance(b,np.ndarray):
    b= np.array(b)
  
  Q, R= QRfactorization(matrixA)
  
  new_b= np.dot(Q.T,b)
  
  x= np.zeros((matrixA.shape[1],1))
  x[-1]= new_b[-1]/R[-1,-1]
  for i in range(matrixA.shape[0]-2,-1,-1):
    s= 0
    for j in range(i+1,matrixA.shape[1]):
      s+= R[i,j]*x[j]
    x[i]= (new_b[i]-s)/R[i,i]
    
  #x= x.tolist()
  #x= [np.round(x[i][0],10) for i in range(0,len(x))]
  
  return x
  

## Extra assignment

###3. Least squares problem $Ax=b$
####Definition
In the section about the direct solver we explored the algorithm for solving systems of linear equations $Ax=b$ where $A$ is a square $n\times n$ matrix. Now, let us consider a system of linear equations $Ax=b$ where, instead, $A$ is a $m\times n$ matrix, with $m>n$ and $b\in\mathbb{R}^m$. Since there exists no inverse matrix $A^{-1}$, we cannot use the previous algorithm to solve $Ax=b$ but we have to apply other algorithms.

We will examine the general case, where $b\notin \text{range}(A)$: for these cases, we say that the system $Ax=b$ is *overdetermined* [6]. Since there exists no exact solution $x$ to the system $Ax=b$, we have to find an approximation of the unknown vector $x$ that solves the system, and one method is called *least squares problem*. 

Least squares problem consists of minimizing the residual $r= b-Ax \in\mathbb{R}^m$, or similarly

>$
\displaystyle x=\arg \min_{y\in\mathbb{R}^n} ||b-Ay||
$

Since $b\notin\text{range}(A)$, looking for $x$ such that minimizes the residual $r=b-Ax$ means that we are seeking a vector $x\in\mathbb{R}^n$ such that the Euclidian distance betweem $Ax$ and $b$ is minimal [6]: this is equal to

>$
A^Tr= 0 \Longleftrightarrow A^TAx=A^Tb
$

And thus, $A^TA$ is always symmetric positive-semidefinite matrix because it has only real entries [7]. If $A^TA$ is invertible, the vector $x$ is given by

>$
  x= \left(A^TA\right)^{-1}A^Tb
$

####Implementation
For the implementation of the least squares problem, we start considering the equation

>$
  A^TAx= A^Tb
$

where

* $D=A^TA\in\mathbb{R}^{n\times n}$
* $x\in\mathbb{R}^n$
* $e=A^Tb\in\mathbb{R}^n$

Consequently, we can rewrite the equation $A^TAx= A^Tb$ as

>$
Dx=e
$

and use the direct solver algorithm presented in the previous section.

In [0]:
def leastSquareProblem(matrixA,b):
  '''
  Solves the system Ax=b by using least square method.
  Parameters:
  - matrixA: m x n matrix, with m > n
  - b: m x 1 vector
  Output:
  - x: n x 1 vector
  '''
  
  if not isinstance(matrixA,np.ndarray):
    matrixA= np.array(matrixA)
    
  if matrixA.shape[0] < matrixA.shape[1]:
    print('Error: too many unknowns! Impossible to solve the system.')
    sys.exit(1)
    
  D= np.dot(matrixA.T,matrixA)                  # D is defined as above
  e= np.dot(matrixA.T,b)                        # e is defined as above
  
  x= linearSystemSolver(D,e)
  
  return x

###4. QR eigenvalue algorithm
####Definition
The eigenvalues for a matrix $A\in\mathbb{R}^{n\times n}$ are defined as the solution of the equation

>$
\det(A-\lambda I)=0
$

which is known as *characteristic equation* for $A$.

The eigenvectors $\{x_i\}$ associated to the $i-$th eigenvalue $\lambda_i$ are defined by 

>$
  \left(A-\lambda_iI\right)x=0
$


####Implementation
One way to seek the eigenvalues and their corresponding eigenvectors for a square $n\times n$ matrix $A$ is to find a matrix $T$ which is similar to $A$ by using similarity transforms. In fact, similar matrices have the same eigenvalues [8,9,10,11].

My implementation consists of two parts:

#####1. QR algorithm 
This algorithm [12,13] is based on Schur factorization of a matrix from successive QR factorizations. The output of this algorithm is a matrix $A^{(k)}$ similar to the input matrix $A$ and it is an upper triangular matrix with eigenvalues on the diagonal. The algorithm is the following one:

```
A_k = A
for k = 1, 2, ... do
  Q_k,R_k= QRfactorization(A_k)
  A_k= R_k * Q_k
end
```



#####2. Rayleigh quotient
After estimating the eigenvalues of matrix $A$, we have to compute the eigenvectors associated to each eigenvalue. The aim of using also this algorithm is to obtain more accurate eigenvalue estimates and at the same time to obtain the estimation of the eigenvector associated to each eigenvalue.

The Rayleigh quotient is an iterative method, where at $i-$th step the approximation of the eigenvector $v_i$ associated to the eigenvalue $\lambda_i$ is defined as

>$
  b_{i}= \dfrac{(A-\mu_{i-1}I)^{-1}b_{i-1}}{||(A-\mu_{i-1}I)^{-1}b_{i-1}||}
$

where $\mu_{i-1}$ is the eigenvalue to the Rayleigh quotient of the current iteration equal to

>$
\mu_{i-1}=\dfrac{b_{i-1}^TAb_{i-1}}{b_{i-1}^tb_{i-1}}
$

As we can see, $(A-\mu_{i-1}I)^{-1}b_{i-1}$ is the solution of the equation $(A-\mu_{i-1}I)x=b_{i-1}$ and thus we can use the direct solver in order to calculate $x= (A-\mu_{i-1}I)^{-1}b_{i-1}$
The algorithm is the following one:

```
b_k = b
mu_k = mu
for k= 1, 2, ... do
  b_k1 = inverse(A-mu_k*I) * b_k
  b_k = b_k1 / norm(b_k1)
  mu_k = (transpose(b_k) * A * b_k) / (norm(b_k)^2)
end
```





In [0]:
def eigenvaluesQRalgorithm(matrixA,convergence=1e-8):
  '''
  Calculates eigenvalues and eigenvectors of the matrixA by using the QR 
  algorithm.
  Parameters:
  - matrixA: n x n matrix
  - convergence (default = 1e-8): limit under which the algorithm stops the 
    iterations
  Output:
  - eigenvalues: eigenvalues of matrixA
  - eigenvectors: eigenvectors of matrixA
  '''
  
  if not isinstance(matrixA,np.ndarray):
    matrixA= np.array(matrixA)
    
  # QR algorithm
  n= matrixA.shape[0]
  A= matrixA
  it= 1
  Q= []
  while it < 2500:
    Q_tmp, R= QRfactorization(A)                           # factorization of A
    A= np.dot(R,Q_tmp)                                     # A = RQ
    if it == 1:
      Q= Q_tmp
    else:
      Q= np.dot(Q,Q_tmp)
    
    it+= 1
    
  eigenval= [A[i,i] for i in range(0,n)]
  eigenvec= []
  
  # Rayleigh quotient
  for i in range(0,len(eigenval)):                         # do the loop for all
                                                           # the eigenvalues
      b= Q[:,i]
      mu= eigenval[i]
      matr= matrixA - mu*np.eye(n)                         # (A-mu*I) as defined
                                                           # above
      r= np.dot(matrixA,b) - mu*b
      r= np.inner(r,r)**(0.5)                              # residual used for 
                                                           # the convergence
      
      iterations= 1
      while r > convergence and iterations < 1000:
        tmp= leastSquareProblem(matr,b)                    # tmp= ||A-mu*b(i-1)||
        if math.isnan(np.inner(tmp,tmp)) or np.inner(tmp,tmp) == 0:
          break
        
        b= tmp/(mp.inner(tmp,tmp)**(0.5))                  # normalization
        mu= np.dot(np.dot(b.T,matrixA),b)/np.inner(b,b)    # new eigenvalue mu
        
        r= np.dot(matrixA,b) - mu*b
        r= np.inner(r,r)**(0.5)
        print('r= ', r)
        iterations+= 1
      
      eigenval[i]= mu
      b= b/(np.inner(b,b)**(0.5))
      eigenvec.append(b)
      
  return eigenval, eigenvec

# **Results**

In this section, we can see the results of the algorithms explained in the previous paragraph in order to verify wether these algorithms work properly or not. It is obviously impossible to test the algorithms with all the possible inputs but here we offer just some examples in order to have a general view for each algorithm.

## QR factorization
Below there are the results for the QR factorization.

In [33]:
def frobeniusNorm(m):
  if not isinstance(m,np.ndarray):
    m= np.array(m)
  new_m= np.dot(m,m.T)
  
  return np.trace(new_m)**(0.5)


count= 0
for i in range(2,27):
  # creating a random upper triangular matrix
  matrix= np.random.randint(1,21,size=(i,i))
  matrix= np.triu(matrix)
  
  Q,R= QRfactorization(matrix)
  flag= True
  flag= flag and np.array_equal(Q,np.eye(i))
  flag= flag and np.array_equal(R,matrix)
  if flag:
    count += 1
    
  print('test {}'.format(i-1))
  print('|| Q^TQ-I ||= ', frobeniusNorm(np.dot(Q.T,Q)-np.eye(i)))
  print('|| QR - A ||= ', frobeniusNorm(np.dot(Q,R)-matrix))
  
    
print(count, '/25 successes')

test 1
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 2
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 3
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 4
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 5
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 6
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 7
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 8
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 9
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 10
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 11
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 12
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 13
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 14
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 15
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 16
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 17
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 18
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 19
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 20
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 21
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0.0
test 22
|| Q^TQ-I ||=  0.0
|| QR - A ||=  0

As we can see, all the tests are been passed.

## Direct solver $Ax=b$
Below there are the results for the direct solver.

In [66]:
print('---------TEST 1----------------------')
matrix= [[1,  1],
         [1, -1]]

b= [[5], 
    [1]]

x= linearSystemSolver(matrix,b)
x_manual= [[3],
           [2]]
x_manual= np.array(x_manual)

print('x.T=', x.T)
print('|| Ax - b || =',np.round(frobeniusNorm(np.dot(matrix,x)-b),10))
print('|| x - x_manual ||=',np.round(frobeniusNorm(x-x_manual),10))
print()

print('---------TEST 2----------------------')
matrix= [[ 2, 4, 6],
         [ 3, 8, 5],
         [-1, 1, 2]]

b= [[22],
    [27],
    [ 2]]
b= np.array(b)

x= linearSystemSolver(matrix,b)
x_manual= [[3],
           [1],
           [2]]
x_manual= np.array(x_manual)

print('x.T=', x.T)
print('|| Ax - b || =',np.round(frobeniusNorm(np.dot(matrix,x)-b),10))
print('|| x - x_manual ||=',np.round(frobeniusNorm(x-x_manual),10))
print()

print('---------TEST 3----------------------')
matrix= [[ 3, -2,  8],
         [-2,  2,  1],
         [ 1,  2, -3]]

b= [[9],
    [3],
    [8]]
b= np.array(b)

x= linearSystemSolver(matrix,b)
x_manual= [[3],
           [4],
           [1]]
x_manual= np.array(x_manual)

print('x.T=', x.T)
print('|| Ax - b || =',np.round(frobeniusNorm(np.dot(matrix,x)-b),10))
print('|| x - x_manual ||=',np.round(frobeniusNorm(x-x_manual),10))
print()

print('---------TEST 4----------------------')
matrix= [[ 1, -2,  2, -3],
         [ 3,  2, -1,  5],
         [ 2, -3,  4, -1],
         [ 3, -2, -1,  2]]

b= [[-7],
    [22],
    [-3],
    [12]]
b= np.array(b)

x= linearSystemSolver(matrix,b)
x_manual= [[ 3],
           [ 1],
           [-1],
           [ 2]]
x_manual= np.array(x_manual)

print('x.T=', x.T)
print('|| Ax - b || =',np.round(frobeniusNorm(np.dot(matrix,x)-b),10))
print('|| x - x_manual ||=',np.round(frobeniusNorm(x-x_manual),10))

---------TEST 1----------------------
x.T= [[3. 2.]]
|| Ax - b || = 0.0
|| x - x_manual ||= 0.0

---------TEST 2----------------------
x.T= [[3. 1. 2.]]
|| Ax - b || = 0.0
|| x - x_manual ||= 0.0

---------TEST 3----------------------
x.T= [[3. 4. 1.]]
|| Ax - b || = 0.0
|| x - x_manual ||= 0.0

---------TEST 4----------------------
x.T= [[ 3.  1. -1.  2.]]
|| Ax - b || = 0.0
|| x - x_manual ||= 0.0


As we can observe, rounding to the 10th decimal place all the residuals are equal to 0. This means that this implementation the results can be considered correct.

All the exercise have been taken from [16].

##Least squares problem
Below there are the results for the least squares problem.

In [90]:
def tester(A,x,b):
  '''
  Tests 100 possible vectors and compare the residuals with the one of the 
  vector x in input. If all the residuals are greater than the residual of the 
  vector x in input, the output is True, otherwise is False.
  '''
  r= frobeniusNorm(np.dot(A,x)-b)
  count = 0
  for _ in range(0,100):
    y= np.random.rand(x.shape[0],x.shape[1])*100
    rr= np.dot(A,y) - b
    rr = frobeniusNorm(rr)
    if r < rr:
      count += 1
    
  if count == 100:
    return '|| Ax - b ||= {:.3f} is the lowest residual'.format(r)
  else:
    return '|| Ax - b ||= {:.3f} is not the lowest residual'.format(r)

print('-----------TEST 1--------------------------')
matrix = [[1, -1],
          [1,  1],
          [2,  1]]

b= [[2],
    [4],
    [8]]

x= leastSquareProblem(matrix,b)
print('x.T=',x.T)
print(tester(matrix,x,b))
print('Exercise taken from [17]\n')

print('-----------TEST 2--------------------------')
matrix= [[ 1,  1,  1],
         [-2,  0, -5],
         [ 1,  2, -1],
         [ 0, -1,  2]]

b= [[-1],
    [ 3],
    [-2],
    [ 1]]

x= leastSquareProblem(matrix,b)
print('x.T=',x.T)
print(tester(matrix,x,b),'\n')

print('-----------TEST 3--------------------------')
matrix= [[ 4,  0,  1,  0],
         [ 0,  2, -1, -2],
         [ 1, -1,  4,  0],
         [ 0, -2,  0,  4],
         [-2,  5,  8, -2],
         [ 3, -9,  0,  1]]

b= [[-1],
    [ 3],
    [-2],
    [ 1],
    [ 0],
    [-8]]

x= leastSquareProblem(matrix,b)
print('x.T=',x.T)
print(tester(matrix,x,b),'\n')

-----------TEST 1--------------------------
x.T= [[3.28571429 1.14285714]]
|| Ax - b ||= 0.535 is the lowest residual
Exercise taken from [17]

-----------TEST 2--------------------------
x.T= [[-9.  5.  3.]]
|| Ax - b ||= 0.000 is the lowest residual 

-----------TEST 3--------------------------
x.T= [[-0.06225645  0.93071731 -0.4495839   0.51975428]]
|| Ax - b ||= 2.083 is the lowest residual 



The tests constist of evaluating whether the vector $x$, resulting from ```leastSquareProblem```, is really the vector for which the residual $r= b- Ax$ is minimal. The tests are done by computing the residual with 100 randomly chosen vectors $y$.



## QR eigenvalue algorithm
Below there are the results of tests on the QR eigenvalue algorithm.

In [105]:
def symmetrize(A):
  if not isinstance(A,np.ndarray):
    A= np.array(A)
    
  return A + A.T

for i in range(2,27):
  print('------------TEST {}-----------------'.format(i-1))
  # creating a random matrix
  matrix= np.random.randint(0,21,size=(i,i))
  matrix= symmetrize(matrix)
  
  eigval, eigvec= eigenvaluesQRalgorithm(matrix) 
  
  flag= True
  for k in range(0,len(eigval)):
    d= linalg.det(matrix - eigval[k]*np.eye(i))
    print('det(A - mu*I)= ',d)
    r= np.dot(matrix,eigvec[k]) - eigval[k]*eigvec[k]
    r= sum(r**2)**(0.5)
    print('|| Av_i - mu_i v_i ||= ', r)
    



------------TEST 1-----------------
det(A - mu*I)=  0.0
|| Av_i - mu_i v_i ||=  3.552713678800501e-15
det(A - mu*I)=  -6.181721801112868e-13
|| Av_i - mu_i v_i ||=  1.4210854715202004e-14
------------TEST 2-----------------
det(A - mu*I)=  8.988785107513152e-11
|| Av_i - mu_i v_i ||=  2.5618982671915014e-14
det(A - mu*I)=  1.0867962924104848e-11
|| Av_i - mu_i v_i ||=  1.5587468005037626e-14
det(A - mu*I)=  0.0
|| Av_i - mu_i v_i ||=  2.730497355672493e-12
------------TEST 3-----------------
det(A - mu*I)=  -3.4948240819966532e-09
|| Av_i - mu_i v_i ||=  4.322062729257126e-14
det(A - mu*I)=  -4.539669443793264e-11
|| Av_i - mu_i v_i ||=  1.4033785549772173e-14
det(A - mu*I)=  -3.7946857163740626e-11
|| Av_i - mu_i v_i ||=  5.052673653147905e-14
det(A - mu*I)=  -9.526989814971288e-12
|| Av_i - mu_i v_i ||=  1.7161993997026529e-13
------------TEST 4-----------------
det(A - mu*I)=  2.9732433019252463e-06
|| Av_i - mu_i v_i ||=  3.8263897345803626e-14
det(A - mu*I)=  -9.632113349819984e-0

TypeError: ignored

# **Discussion**

Summarize your results and your conclusions. Were the results expected or surprising. Do your results have implications outside the particular problem investigated in this report? 

# **References**

[1] from Wikipedia, *[Direct methods](https://en.wikipedia.org/wiki/Direct_method_in_the_calculus_of_variations)*

[2] from Wikipedia, *[QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition)*

[3] from Lecture Notes, *Classical Gram-Schmidt orthogonalization, chapter 5*, pg. 69

[4] from Lecture Notes, *Modified Gram-Schmidt orthogonalization, chapter 5*, pg. 70

[5] from Lecture Notes, *Triangular matrices, chapter 5*, pg. 69

[6] from Lecture Notes, *Least square problems, chapter 5*, pg. 74

[7] from Wikipedia, *[Transpose](https://en.wikipedia.org/wiki/Transpose)*

[8] from Lecture Notes, *Theorem 7, chapter 6*, pg. 80

[9] from Lecture Notes, *Theorem 8, chapter 6,* pg. 81

[10] from Lecture Notes, *Theorem 9, chapter 6,* pg. 81

[11] from Lecture Notes, *Theorem 10, chapter 6*, pg. 81

[12] from YouTube, *[QR algorithm for eigenvalues](https://www.youtube.com/watch?v=_neGVEBjLJA)*

[13] from Lecture Note, *QR algoritm, chapter 6*, pg. 82

[14] from Lecture Note, *Rayleigh quotation, chapter 6*, pg. 83

[15] from Wikipedia, *[Rayleigh quotation](https://en.wikipedia.org/wiki/Rayleigh_quotient_iteration)*

[16] *[Exercises](http://www.math.tamu.edu/~shatalov/Tan_chpt2.pdf)*

[17] *[Exercise](https://s-mat-pcs.oulu.fi/~mpa/matreng/eem5_5-1.htm)*